In [ ]:
# Purpose: Aggregate Individual Predictions from Elastic Net, GBT, and both implementations of Random Forest
# Inputs: Predictions from Lasso RF, Weighted Multiple Random Forest, Elastic Net, and Gradient-boosted Tree
# Outputs: Simple Team Average, ad-hoc Weighted Average (from LB performance), best Linear Regression Ensemble, best Random Forest Ensemble
# Machine: High-performance Cluster (64 cores), ~1 hr

In [9]:
# Packages Used
import pandas as pd
import numpy as np
import scipy.stats as sp
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import brier_score_loss, mean_squared_error
from scipy.stats import randint, uniform ### IMPORTANT: these are distributions and not draws
from sklearn.model_selection import cross_val_score, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics.scorer import make_scorer
import copy
%matplotlib inline

In [ ]:
# Random Seed
np.random.seed(0)

## Average predictions

In [2]:
df_abdul = pd.read_csv('../output/final_pred/weighted_multiRF_prediction.csv',index_col='challengeID')
df_dan = pd.read_csv('../output/final_pred/lassoRF_prediction.csv',index_col='challengeID')
df_eaman = pd.read_csv('../output/final_pred/elastic_prediction.csv',index_col='challengeID')
df_yoshi = pd.read_csv('../output/final_pred/xgboost_prediction.csv',index_col='challengeID')

In [3]:
average_prediction = (df_abdul + df_dan + df_eaman+df_yoshi)/4
average_prediction.eviction = (df_abdul.eviction+df_dan.eviction+df_yoshi.eviction)/3
average_prediction.layoff = (df_abdul.layoff+df_dan.layoff+df_yoshi.layoff)/3
average_prediction.jobTraining = (df_yoshi.jobTraining + df_dan.jobTraining+df_abdul.jobTraining)/3.

average_prediction.to_csv('../output/final_pred/avrg_prediction.csv')

In [4]:
## Weighted Avg Predictions

In [5]:
weighted_prediction = (3*df_abdul + 2*df_eaman + df_yoshi)/6
weighted_prediction.gpa = (3*df_eaman.gpa + 2*df_abdul.gpa + df_dan.gpa)/6
weighted_prediction.grit = (3*df_eaman.grit + 2*df_dan.grit + df_abdul.grit)/6
weighted_prediction.layoff = (2*df_abdul.layoff + df_dan.layoff)/3
weighted_prediction.eviction = (2*df_abdul.eviction + df_dan.eviction)/3
weighted_prediction.jobTraining = (3*df_abdul + 2*df_yoshi + df_dan)/6

weighted_prediction.to_csv('../output/final_pred/weighted_avrg_prediction.csv')

## Ensemble (Linear Regression)

In [11]:
for col in df_abdul.columns:
    df_abdul[col+'_abdul'] = df_abdul[col]
    del df_abdul[col]
    
for col in df_dan.columns:
    df_dan[col+'_dan'] = df_dan[col]
    del df_dan[col]
    
for col in df_eaman.columns:
    df_eaman[col+'_eaman'] = df_eaman[col]
    del df_eaman[col]

dfx = df_abdul.join(df_eaman).join(df_dan)
dfy = pd.read_csv('../data/train.csv',index_col='challengeID')
predictions = {'challengeID':np.array(list(dfx.index)),
               'gpa':None,'grit':None,'materialHardship':None,'eviction':None,'layoff':None,'jobTraining':None}  
outcomes = list(dfy.columns) #get the names of the outcomes

In [7]:
NUM_MODELS = 10
n_CVjobs = 9
n_CVsplits = 5
n_modelJobs = 7 #*4 ##remove the comment on EC2
mode = None
n_iter_search = 30
max_features = 18 ##this should be < n_features

reg_outcomes = ['gpa', 'grit', 'materialHardship']
clf_outcomes = [ 'eviction', 'layoff', 'jobTraining']




__reg_param_dist = {"fit_intercept":[True,False], "normalize":[True,False]}

__clf_param_dist = { "C":[0.001,0.00001,1.0,10], "fit_intercept":[True,False]}


###### We don't use this anymore (where we average the parameters of the model)####
#__reg_param = {'max_depth': [],
#               'max_features': [],
#               'min_samples_split':[],
#               'min_samples_leaf':[],
#               'n_estimators':[],
#               'oob_score':[]}
#
#__clf_param = {'max_depth': [],
#               'max_features': [],
#               'min_samples_split':[],
#               'min_samples_leaf':[],
#               'n_estimators':[],
#               'criterion':[]}
#best_param = {'reg' : __reg_param,
#              'clf': __clf_param}

param_dist = {'reg' : __reg_param_dist,
              'clf': __clf_param_dist}

model = {'reg' : LinearRegression(),
          'clf': LogisticRegression()}

scorer = {'reg' : make_scorer(mean_squared_error,greater_is_better=False),
           'clf' : make_scorer(brier_score_loss,greater_is_better=False)}

evaluate_error = {'reg': mean_squared_error,
                  'clf': brier_score_loss}


best_model_prediction = {'challengeID':np.array(list(dfx.index)),
               'gpa':None,
               'grit':None,
               'materialHardship':None,
               'eviction':None,
               'layoff': None,
               'jobTraining':None
              }

avg_models_prediction = copy.deepcopy(best_model_prediction)
weighted_models_prediction = copy.deepcopy(best_model_prediction)



for outcome in outcomes:
    ##Figure out in what mode we are
    if outcome in reg_outcomes:
        mode = 'reg'
    else:
        mode = 'clf'
    
    ###prepare X and Y####
    full = dfx.join(dfy, how='outer') #connect the background data to outcomes
    full_X = full.copy()
    for inner_outcome in outcomes:
        del full[inner_outcome]
    X = full_X.dropna(subset=[outcome], how='all')
    y = X[outcome]
    for inner_outcome in outcomes:
        del full_X[inner_outcome]

    for inner_outcome in outcomes:
        del X[inner_outcome]
        
    ##In order to try the different aggregation mechanisms
    combined_model_prediction = {'challengeID':np.array(list(dfx.index)),outcome: None}
    lowest_error = np.inf
    best_model = None
    all_models_scores = []
    weighted_models = [] 
    n_good_models = 0

    for i in range(1,NUM_MODELS+1):
        print('at loop:',i,'for outcome ', outcome)
        ##prepare the nested CV
        inner_cv = StratifiedKFold(n_splits=n_CVsplits, shuffle=True, random_state=i)
        outer_cv = StratifiedKFold(n_splits=n_CVsplits, shuffle=True, random_state=i)

        ########Nested CV with parameter optimization########
        #the Randomized search for the best parameters through cross validation
        search = GridSearchCV(estimator=model[mode], param_grid=param_dist[mode],
                                    scoring = scorer[mode],
                                    cv=inner_cv,n_jobs=n_CVjobs)
        search.fit(X, y)
        #The evaluation of the best model found by the inner CV by having an outer CV
        nested_score = cross_val_score(search, X=X, y=y, cv=outer_cv)
        if mode == 'reg':
            prediction = search.best_estimator_.predict(X)
        else:
            prediction = search.best_estimator_.predict_proba(X)[:,1]

        if evaluate_error[mode](y,prediction) < np.inf:
            n_good_models +=1

            if evaluate_error[mode](y,prediction) < lowest_error:
                print('best so far')
                lowest_error = evaluate_error[mode](y,prediction)
                best_model = search.best_estimator_

            if i == 1:
                if mode == 'reg':
                    combined_model_prediction[outcome] = search.best_estimator_.predict(full_X)
                else:
                    combined_model_prediction[outcome] = search.best_estimator_.predict_proba(full_X)[:,1]
            else:
                if mode == 'reg':
                    combined_model_prediction[outcome] = combined_model_prediction[outcome] + search.best_estimator_.predict(full_X)
                else:
                    combined_model_prediction[outcome] = combined_model_prediction[outcome]+ search.best_estimator_.predict_proba(full_X)[:,1]

                

            all_models_scores.append(evaluate_error[mode](y,prediction))
            if mode == 'reg':
                weighted_models.append(search.best_estimator_.predict(full_X))
            else:
                weighted_models.append(search.best_estimator_.predict_proba(full_X)[:,1])
        print('score:', evaluate_error[mode](y,prediction))
        print('CV scores:', nested_score.mean(), nested_score)
        print('best params', search.best_params_)
        print('#######')
    

        
        ##best model prediction
        best_model.fit(X, y)
        if mode == 'reg':
            final_prediction = best_model.predict(full_X)
        else:
            final_prediction = best_model.predict_proba(full_X)[:,1]
        best_model_prediction[outcome] = final_prediction
        
        ##avg models prediction
        avg_models_prediction[outcome] = combined_model_prediction[outcome]/float(n_good_models)
        
        
        ##weighted models prediction
        scores = np.array(all_models_scores)
        normlized_scores = scores/sum(scores)
        normlized_scores = normlized_scores.reshape(normlized_scores.shape[0],1)
        models_predicitons = np.matrix(weighted_models).T
        weighted_prediction = np.array(models_predicitons*normlized_scores).flatten().tolist()
        weighted_models_prediction[outcome] = weighted_prediction

df_best = pd.DataFrame.from_dict(best_model_prediction)
df_avg = pd.DataFrame.from_dict(avg_models_prediction)
df_weighted = pd.DataFrame.from_dict(weighted_models_prediction)

df_best.to_csv('../output/final_pred/LR_prediction.csv')
#df_avg.to_csv('./Linear_avg_prediction.csv',index=False)
#df_weighted.to_csv('./Linear_weighted_prediction.csv',index=False)

at loop: 1 for outcome  gpa
best so far
score: 0.0124570426475
CV scores: -1.75089160221 [-0.0134401  -0.01361169 -8.7041309  -0.01238989 -0.01088542]
best params {'fit_intercept': True, 'normalize': False}
#######
at loop: 2 for outcome  gpa
score: 0.0124570426475
CV scores: -0.0128019691045 [-0.01294373 -0.01404033 -0.01340153 -0.01172618 -0.01189808]
best params {'fit_intercept': False, 'normalize': True}
#######
at loop: 3 for outcome  gpa
score: 0.0124570426475
CV scores: -0.0129101445572 [-0.01225124 -0.01357717 -0.01156618 -0.01356161 -0.01359453]
best params {'fit_intercept': False, 'normalize': True}
#######
at loop: 4 for outcome  gpa
score: 0.0124570426475
CV scores: -0.0129199912603 [-0.01379953 -0.01133819 -0.01282343 -0.01318793 -0.01345087]
best params {'fit_intercept': False, 'normalize': True}
#######
at loop: 5 for outcome  gpa
score: 0.0124570426475
CV scores: -0.0128032952029 [-0.01333758 -0.01480875 -0.01242337 -0.01071327 -0.01273351]
best params {'fit_intercept':

score: 0.0143468949357
CV scores: -0.0198562096103 [-0.02389078 -0.03071672 -0.01718213 -0.01718213 -0.01030928]
best params {'C': 1.0, 'fit_intercept': True}
#######
at loop: 1 for outcome  layoff
best so far
score: 0.0395700665593
CV scores: -0.0579258578431 [-0.06640625 -0.078125   -0.05882353 -0.03921569 -0.04705882]
best params {'C': 10, 'fit_intercept': True}
#######
at loop: 2 for outcome  layoff
score: 0.0395700665593
CV scores: -0.0571691176471 [-0.07421875 -0.03515625 -0.05490196 -0.05882353 -0.0627451 ]
best params {'C': 10, 'fit_intercept': True}
#######
at loop: 3 for outcome  layoff
score: 0.039684323691
CV scores: -0.0564154411765 [-0.03125    -0.0390625  -0.08235294 -0.0627451  -0.06666667]
best params {'C': 10, 'fit_intercept': False}
#######
at loop: 4 for outcome  layoff
score: 0.0395700665593
CV scores: -0.056393995098 [-0.03515625 -0.0625     -0.05098039 -0.07058824 -0.0627451 ]
best params {'C': 10, 'fit_intercept': True}
#######
at loop: 5 for outcome  layoff
sco

## Ensemble (Random Forest)

In [20]:
NUM_MODELS = 10
n_CVjobs = -1
n_CVsplits = 5
n_modelJobs = -1 #*4 ##remove the comment on EC2
mode = None
n_iter_search = 30
max_features = 15 ##this should be the number of features / 5

reg_outcomes = ['gpa', 'grit', 'materialHardship']
clf_outcomes = [ 'eviction', 'layoff', 'jobTraining']




__reg_param_dist = {'max_depth': [1,2,3,4,None],
                    'max_features': randint(1, max_features),
                    'min_samples_split':randint(2, 300),
                    'min_samples_leaf':randint(1, 300),
                    'n_estimators':randint(50, 500),
                    'oob_score':[True,False]}

__clf_param_dist = {'max_depth': [1,2,3,4,None],
                    'max_features': randint(1, max_features),
                    'min_samples_split':randint(2, 300),
                    'min_samples_leaf':randint(1, 300),
                    'n_estimators':randint(50, 500),
                    'criterion':['gini','entropy']}


###### We don't use this anymore (where we average the parameters of the model)####
#__reg_param = {'max_depth': [],
#               'max_features': [],
#               'min_samples_split':[],
#               'min_samples_leaf':[],
#               'n_estimators':[],
#               'oob_score':[]}
#
#__clf_param = {'max_depth': [],
#               'max_features': [],
#               'min_samples_split':[],
#               'min_samples_leaf':[],
#               'n_estimators':[],
#               'criterion':[]}
#best_param = {'reg' : __reg_param,
#              'clf': __clf_param}

param_dist = {'reg' : __reg_param_dist,
              'clf': __clf_param_dist}

model = {'reg' : RandomForestRegressor(n_jobs=n_modelJobs),
          'clf': RandomForestClassifier(n_jobs=n_modelJobs)}

scorer = {'reg' : make_scorer(mean_squared_error,greater_is_better=False),
           'clf' : make_scorer(brier_score_loss,greater_is_better=False)}

evaluate_error = {'reg': mean_squared_error,
                  'clf': brier_score_loss}


best_model_prediction = {'challengeID':np.array(list(dfx.index)),
               'gpa':None,
               'grit':None,
               'materialHardship':None,
               'eviction':None,
               'layoff': None,
               'jobTraining':None
              }

avg_models_prediction = copy.deepcopy(best_model_prediction)
weighted_models_prediction = copy.deepcopy(best_model_prediction)


for outcome in outcomes:
    ##Figure out in what mode we are
    if outcome in reg_outcomes:
        mode = 'reg'
    else:
        mode = 'clf'
    
    ###prepare X and Y####
    full = dfx.join(dfy, how='outer') #connect the background data to outcomes
    full_X = full.copy()
    for inner_outcome in outcomes:
        del full[inner_outcome]
    X = full_X.dropna(subset=[outcome], how='all')
    y = X[outcome]
    for inner_outcome in outcomes:
        del full_X[inner_outcome]

    for inner_outcome in outcomes:
        del X[inner_outcome]
        
    ##In order to try the different aggregation mechanisms
    combined_model_prediction = {'challengeID':np.array(list(dfx.index)),outcome: None}
    lowest_error = np.inf
    best_model = None
    all_models_scores = []
    weighted_models = [] 
    n_good_models = 0

    for i in range(1,NUM_MODELS+1):
        print('at loop:',i,'for outcome ', outcome)
        ##prepare the nested CV
        inner_cv = StratifiedKFold(n_splits=n_CVsplits, shuffle=True, random_state=i)
        outer_cv = StratifiedKFold(n_splits=n_CVsplits, shuffle=True, random_state=i)

        ########Nested CV with parameter optimization########
        #the Randomized search for the best parameters through cross validation
        search = RandomizedSearchCV(estimator=model[mode], param_distributions=param_dist[mode],
                                    scoring = scorer[mode],
                                    cv=inner_cv,n_jobs=n_CVjobs,n_iter=n_iter_search)
        search.fit(X, y)
        #The evaluation of the best model found by the inner CV by having an outer CV
        nested_score = cross_val_score(search, X=X, y=y, cv=outer_cv)
        if mode == 'reg':
            prediction = search.best_estimator_.predict(X)
        else:
            prediction = search.best_estimator_.predict_proba(X)[:,1]

        if evaluate_error[mode](y,prediction) < np.inf:
            n_good_models +=1

            if evaluate_error[mode](y,prediction) < lowest_error:
                print('best so far')
                lowest_error = evaluate_error[mode](y,prediction)
                best_model = search.best_estimator_

            if i == 1:
                if mode == 'reg':
                    combined_model_prediction[outcome] = search.best_estimator_.predict(full_X)
                else:
                    combined_model_prediction[outcome] = search.best_estimator_.predict_proba(full_X)[:,1]
            else:
                if mode == 'reg':
                    combined_model_prediction[outcome] = combined_model_prediction[outcome] + search.best_estimator_.predict(full_X)
                else:
                    combined_model_prediction[outcome] = combined_model_prediction[outcome]+ search.best_estimator_.predict_proba(full_X)[:,1]

                

            all_models_scores.append(evaluate_error[mode](y,prediction))
            if mode == 'reg':
                weighted_models.append(search.best_estimator_.predict(full_X))
            else:
                weighted_models.append(search.best_estimator_.predict_proba(full_X)[:,1])
        print('score:', evaluate_error[mode](y,prediction))
        print('CV scores:', nested_score.mean(), nested_score)
        print('best params', search.best_params_)
        print('#######')
    

        
        ##best model prediction
        best_model.fit(X, y)
        if mode == 'reg':
            final_prediction = best_model.predict(full_X)
        else:
            final_prediction = best_model.predict_proba(full_X)[:,1]
        best_model_prediction[outcome] = final_prediction
        
        ##avg models prediction
        avg_models_prediction[outcome] = combined_model_prediction[outcome]/float(n_good_models)
        
        
        ##weighted models prediction
        scores = np.array(all_models_scores)
        normlized_scores = scores/sum(scores)
        normlized_scores = normlized_scores.reshape(normlized_scores.shape[0],1)
        models_predicitons = np.matrix(weighted_models).T
        weighted_prediction = np.array(models_predicitons*normlized_scores).flatten().tolist()
        weighted_models_prediction[outcome] = weighted_prediction

        



df_best = pd.DataFrame.from_dict(best_model_prediction)
df_avg = pd.DataFrame.from_dict(avg_models_prediction)
df_weighted = pd.DataFrame.from_dict(weighted_models_prediction)

df_best.to_csv('../output/final_pred/RF_prediction.csv')
#df_avg.to_csv(output_file+'/avg_prediction/prediction.csv')
#df_weighted.to_csv(output_file+'/weighted_prediction/prediction.csv')

at loop: 1 for outcome  gpa


KeyboardInterrupt: 